# Scrapes lyrics from billboard hot 100.
Code modified from Kaylin Pavlik using chatGPT to convert from R to Python

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# SCRAPE SONG NAMES
allthesongs = pd.DataFrame(columns=["Rank", "Song", "Artist", "Year", "Lyrics", "Source"])

for i in range(1965,1966):
    URL = f"http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{i}"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.select('table.wikitable.sortable tr')[1:]

    billboard_data = []
    for row in rows:
        columns = row.find_all(['td', 'th'])
        #HERE, I changed strip= true to strip = false to preserve line breaks
        billboard_data.append([col.get_text(strip=False) for col in columns])

    year_data = pd.DataFrame(billboard_data, columns=["Rank", "Song", "Artist"])
    year_data["Year"] = i
    allthesongs = pd.concat([allthesongs, year_data], ignore_index=True)

allthesongs["Song"] = allthesongs["Song"].str.replace('\"', '').str.lower().str.replace("[^a-zA-Z0-9 ']", '')
allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace("[^a-zA-Z0-9 ? & ' -]", '')
allthesongs["Artist"] = allthesongs["Artist"].str.replace("'e", "e").str.replace("'o", "o")
allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace('featuring', ' feat ')


# SCRAPE THE LYRICS
for s in range(len(allthesongs["Song"])):
    lyrics = "Not set yet."
    results = 12  # arbitrary number

    pre_artist = allthesongs["Artist"].iloc[s]
    if pre_artist == None:
      pre_artist = allthesongs["Artist"][s-1]
      print("DONE")
    artist = re.sub(" and ", " & ", pre_artist)
    artist = re.sub("featuring", " feat ", artist)
    pre_artist = artist
    artist2 = re.sub("the ", "", artist)
    first_letter = artist2[0]

    # searches through song urls to find lyrics on the web
    song_url2 = f"http://songlyrics.com/{artist}/{allthesongs['Song'].iloc[s]}-lyrics"
    song_url3 = f"http://songlyrics.com/{pre_artist}/{allthesongs['Song'].iloc[s]}-lyrics"
    song_url = f"http://songlyrics.com/{artist2}/{allthesongs['Song'].iloc[s]}-lyrics"
    mode_url = f"http://www.lyricsmode.com/lyrics/{first_letter}/{artist2}/{allthesongs['Song'].iloc[s]}.html"


    urls = [song_url3, song_url2, song_url, mode_url]
    lyric_locs = ['p#songLyricsDiv', 'p#songLyricsDiv', 'p#songLyricsDiv', 'p#lyrics_text']

    for b, URL in enumerate(urls):
        allthesongs.at[s, "Lyrics"] = "Not set yet."
        results = 12  # arbitrary number

        if b != 3:
          URL = URL.replace(" ", "-").lower()
          URL = URL.replace("\n/", "/")

        try:
            response = requests.get(URL)
            soup = BeautifulSoup(response.text, 'lxml')
            lyrics = soup.select_one(lyric_locs[b]).get_text()
        except Exception as e:
            print(f"{s} failed")
        else:
            if lyrics:
                allthesongs.at[s, "Lyrics"] = lyrics
                print(f"{s} success")
                print(URL)
                allthesongs.at[s, "Source"] = b
                break

allthesongs["Lyrics"] = allthesongs["Lyrics"].str.lower()
missing = round((allthesongs["Lyrics"] == "not set yet").sum() / len(allthesongs) * 100, 2)
allthesongs["Lyrics"] = allthesongs["Lyrics"].replace({"not set yet": "NA",
                                                       "we are not in a position to display these lyrics due to licensing restrictions sorry for the inconvenience": "NA"})

# Set working directory and write to CSV if needed
allthesongs.to_csv("billboard_lyrics.csv", index=False)
allthesongs = pd.read_csv("billboard_lyrics.csv")


Sam the Sham and the Pharaohs

sam the sham and the pharaohs

wooly bully
0 success
http://songlyrics.com/sam-the-sham-&-the-pharaohs/wooly-bully-lyrics
1 success
http://songlyrics.com/four-tops/i-can't-help-myself-(sugar-pie-honey-bunch)-lyrics
2 success
http://songlyrics.com/the-rolling-stones/(i-can't-get-no)-satisfaction-lyrics
3 success
http://songlyrics.com/we-five/you-were-on-my-mind-lyrics
4 success
http://songlyrics.com/the-righteous-brothers/you've-lost-that-lovin'-feelin'-lyrics
5 success
http://songlyrics.com/petula-clark/downtown-lyrics
6 success
http://songlyrics.com/the-beatles/help!-lyrics
7 success
http://songlyrics.com/herman's-hermits/can't-you-hear-my-heartbeat-lyrics
8 success
http://songlyrics.com/elvis-presley/crying-in-the-chapel-lyrics
9 success
http://songlyrics.com/the-temptations/my-girl-lyrics
10 success
http://songlyrics.com/the-beach-boys/help-me,-rhonda-lyrics
11 success
http://songlyrics.com/roger-miller/king-of-the-road-lyrics
12 success
http://songlyr